In [ ]:
import os
from os.path import join
import json
from sklearn.preprocessing import LabelEncoder
os.environ["WORKING_DIR"] = "D:\\semantic_data_lake\\semantic_data_lake"
os.environ["TYPENAME"] = "type_gittables_abstraction"

In [ ]:
valid_types_path = join(os.environ["WORKING_DIR"], "data", "extract", "out", "valid_types", "types.json")
# load array of valid types
with open(valid_types_path, "r") as file:
    valid_types = json.load(file)[os.environ["TYPENAME"]]

In [ ]:
print(len(valid_types))
# build a Label Encoder 
label_enc = LabelEncoder()
label_enc.fit(valid_types)

In [ ]:
label_enc.inverse_transform([0])

In [ ]:
df = pd.DataFrame([["test", "[1,2,3]"]], columns=["0", "1"])
df["1"] = df["1"].apply(eval)
def label_enc_f(x):
    return label_enc.inverse_transform(x)
df["1"] = df["1"].apply(label_enc_f)
df

In [ ]:
%run data_split_labeled_unlabeled_test_absolut.py --valid_headers gittables-abstraction-tables_type_gittables_abstraction_header_valid.csv --corpus gittables-abstraction-tables --labeled_size 1 --test_size 0.2

In [ ]:
import numpy as np
import os
os.environ["TYPENAME"] = "type_turl"

for value in np.arange(1,6,1):
    for random_state in [1,2,3,4,5]:
        %run data_split_labeled_unlabeled_test_absolut.py --valid_headers turl_type_turl_header_valid.csv --corpus turl --labeled_size {value} --test_size 0.2 --random_state {random_state}

In [1]:
import numpy as np
import os
os.environ["TYPENAME"] = "type_turl"

for value in np.around(np.arange(0.002,0.022,0.002),3):
    print(value)
    for random_state in [1,2,3,4,5]:
        %run data_split_labeled_unlabeled_test.py --valid_headers turl_type_turl_header_valid.csv --corpus turl --labeled_size {value} --unlabeled_size {0.80-value} --test_size 0.2 --random_state {random_state}

0.002
Labeled data: 1048(0.00), Unlabeled data: 418272(0.80), Test data: 104831(0.20)
Labeled data: 1048(0.00), Unlabeled data: 418272(0.80), Test data: 104831(0.20)
Labeled data: 1048(0.00), Unlabeled data: 418272(0.80), Test data: 104831(0.20)
Labeled data: 1048(0.00), Unlabeled data: 418272(0.80), Test data: 104831(0.20)
Labeled data: 1048(0.00), Unlabeled data: 418272(0.80), Test data: 104831(0.20)
0.004
Labeled data: 2096(0.00), Unlabeled data: 417224(0.80), Test data: 104831(0.20)
Labeled data: 2096(0.00), Unlabeled data: 417224(0.80), Test data: 104831(0.20)
Labeled data: 2096(0.00), Unlabeled data: 417224(0.80), Test data: 104831(0.20)
Labeled data: 2096(0.00), Unlabeled data: 417224(0.80), Test data: 104831(0.20)
Labeled data: 2096(0.00), Unlabeled data: 417224(0.80), Test data: 104831(0.20)
0.006
Labeled data: 3144(0.01), Unlabeled data: 416176(0.79), Test data: 104831(0.20)
Labeled data: 3144(0.01), Unlabeled data: 416176(0.79), Test data: 104831(0.20)
Labeled data: 3144(0.0

In [ ]:
# transform sato csv valid header format into embclus json valid header format
import json
from sklearn.preprocessing import LabelEncoder
import pandas as pd

with open("./out/valid_types/types.json", "r") as f:
    valid_types = json.load(f)
    valid_types = valid_types["type_turl"]

label_enc = LabelEncoder()
label_enc.fit(valid_types)

#label_enc.inverse_transform([0])

dic = {}
df = pd.read_csv("./out/valid_headers/turl_type_turl_header_valid.csv")
for index, row in df.iterrows():
    dataset_id = row["dataset_id"]
    field_list = eval(row["field_list"])
    field_names = eval(row["field_names"])
    dic[dataset_id] = {}
    for index_2, column in enumerate(field_list):
        dic[dataset_id][f"column_{column}"] = {
            "semanticType": label_enc.inverse_transform([field_names[index_2]])[0]
        }


In [ ]:
with open("./out/valid_headers/turl_type_turl_header_valid.json", "w") as f:
    json.dump(dic,f)

# plot stats of splitting

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from os.path import join
import os

from multiprocesspandas import applyparallel

os.environ["WORKING_DIR"] = "D:\\semantic_data_lake\\semantic_data_lake"

valid_header_path = join(os.environ["WORKING_DIR"], "data", "extract", "out",
                         "valid_headers")

labeled = 1.0
test = 20.0
unlabeled = 100-test-labeled
#unlabeled = "absolute"

valid_headers_file = f"turl_type_turl_valid.json"

# load the valid headers with read sem. types
valid_headers = join(valid_header_path, valid_headers_file)


with open(valid_headers, "r") as file:
    valid_headers = json.load(file)

# transform valid header into df to make splitable
valid_header_df_data = []
for table in valid_headers.keys():
    for column in valid_headers[table].keys():
        valid_header_df_data.append(
            [table, column, valid_headers[table][column]["semanticType"]])
valid_header_df = pd.DataFrame(valid_header_df_data,
                            columns=["table", "column", "semanticType"])

valid_header_df["dataset_id"] = valid_header_df["table"]+"+"+valid_header_df["column"]
print(valid_header_df.head())


#valid_header_df = valid_header_df.iloc[:500]

# load laebeled_unlabeled_test_split
split_file_path = join(os.environ["WORKING_DIR"], "data", "extract", "out", "labeled_unlabeled_test_split")
split_file = f"turl_{labeled}_{unlabeled}_{test}.json"

with open(join(split_file_path, split_file), "r") as file:
    split_file = json.load(file)

df_split_labeled = pd.DataFrame({"dataset_id": split_file[f"labeled{labeled}"]})
df_split_labeled["split_part"] = "labeled"
df_split_unlabeled = pd.DataFrame({"dataset_id": split_file[f"unlabeled{unlabeled}"]})
#df_split_unlabeled = pd.DataFrame({"dataset_id": split_file[f"unlabeled"]})
df_split_unlabeled["split_part"] = "unlabeled"
df_split_test = pd.DataFrame({"dataset_id": split_file[f"test{test}"]})
df_split_test["split_part"] = "test"

df_split = pd.concat([df_split_labeled, df_split_unlabeled, df_split_test])

#df_split.head(-20)

df = valid_header_df.join(df_split.set_index("dataset_id"), on="dataset_id")

save_path = join(os.environ["WORKING_DIR"], "data", "extract", "out", "labeled_unlabeled_test_split")

plt.figure(figsize=(10,30))

ax = df[df["split_part"] == "labeled"].groupby(["semanticType"]).size().sort_values().plot.barh()

for container in ax.containers:
    ax.bar_label(container)

plt.grid()
plt.title(f"#semantic types of labaled data ({labeled}/{unlabeled}/{test})")
plt.savefig(join(save_path, f"turl_{labeled}_{unlabeled}_{test}_stats_labeled.png"), bbox_inches="tight", facecolor="white", transparent=False)
#plt.show()

plt.figure(figsize=(10,30))

ax = df[df["split_part"] == "unlabeled"].groupby(["semanticType"]).size().sort_values().plot.barh()

for container in ax.containers:
    ax.bar_label(container)

plt.grid()
plt.title(f"#semantic types of unlabaled data ({labeled}/{unlabeled}/{test})")
plt.savefig(join(save_path, f"turl_{labeled}_{unlabeled}_{test}_stats_unlabeled.png"), bbox_inches="tight", facecolor="white", transparent=False)
#plt.show()

plt.figure(figsize=(10,30))

ax = df[df["split_part"] == "test"].groupby(["semanticType"]).size().sort_values().plot.barh()

for container in ax.containers:
    ax.bar_label(container)

plt.grid()
plt.title(f"#semantic types of test data ({labeled}/{unlabeled}/{test})")
plt.savefig(join(save_path, f"turl_{labeled}_{unlabeled}_{test}_stats_test.png"), bbox_inches="tight", facecolor="white", transparent=False)
#plt.show()

In [ ]:
labels, counts = np.unique(valid_header_df["semanticType"].tolist(),return_counts=True)
pd.DataFrame({"lables":labels, "counts":counts}).sort_values(by="counts", ascending=False)

In [ ]:
save_path = join(os.environ["WORKING_DIR"], "data", "extract", "out", "labeled_unlabeled_test_split")

plt.figure(figsize=(10,30))

ax = df[df["split_part"] == "labeled"].groupby(["semanticType"]).size().sort_values().plot.barh()

for container in ax.containers:
    ax.bar_label(container)

plt.grid()
plt.title(f"#semantic types of labaled data ({labeled}/{unlabeled}/{test})")
plt.savefig(join(save_path, f"turl_{labeled}_{unlabeled}_{test}_stats_labeled.png"), bbox_inches="tight", facecolor="white", transparent=False)
#plt.show()

plt.figure(figsize=(10,30))

ax = df[df["split_part"] == "unlabeled"].groupby(["semanticType"]).size().sort_values().plot.barh()

for container in ax.containers:
    ax.bar_label(container)

plt.grid()
plt.title(f"#semantic types of unlabaled data ({labeled}/{unlabeled}/{test})")
plt.savefig(join(save_path, f"turl_{labeled}_{unlabeled}_{test}_stats_unlabeled.png"), bbox_inches="tight", facecolor="white", transparent=False)
#plt.show()

plt.figure(figsize=(10,30))

ax = df[df["split_part"] == "test"].groupby(["semanticType"]).size().sort_values().plot.barh()

for container in ax.containers:
    ax.bar_label(container)

plt.grid()
plt.title(f"#semantic types of test data ({labeled}/{unlabeled}/{test})")
plt.savefig(join(save_path, f"turl_{labeled}_{unlabeled}_{test}_stats_test.png"), bbox_inches="tight", facecolor="white", transparent=False)
#plt.show()

In [ ]:
import numpy as np
for labeled in np.around(np.arange(0.1,1,0.1),1):
    %run plot_stats_labeled_unlabeled_test_split.py --labeled {labeled}  --unlabeled {80-labeled} --test 20 --corpus "turl"

In [13]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from os.path import join
import os

os.environ["WORKING_DIR"] = "D:\\semantic_data_lake\\semantic_data_lake"

# valid_header_path = join(os.environ["WORKING_DIR"], "data", "extract", "out",
#                          "valid_headers", "turl_time.event")
# df_valid_header = pd.read_csv(join(valid_header_path, f"turl_type_turl_header_valid.csv"))

valid_header_path = join(os.environ["WORKING_DIR"], "data", "extract", "out",
                         "valid_headers")
df_valid_header = pd.read_csv(join(valid_header_path, f"public_bi_type78.csv"))

headers = []
column_numbers = []
for index, row in df_valid_header.iterrows():
    headers.extend(eval(row["field_names"]))
    column_numbers.append(len(eval(row["field_names"])))


In [ ]:
labels, counts  = np.unique(headers, return_counts=True)
pd.DataFrame({"labels":label_enc.inverse_transform(labels),"counts":counts }).sort_values(by="counts", ascending=False)

In [15]:
import statistics

statistics.mean(column_numbers)

8.955974842767295

In [14]:
sum(column_numbers)

1424